<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/2_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 監聽 Markdown 連結點擊並觸發下載

In [ ]:
def generate_file_links(file_string):
    if not file_string:
        return ""

    files = file_string.split(', ')
    links = []

    for file in files:
        parts = file.split('_', 1)
        if len(parts) > 1:
            display_name = parts[1]
            month = parts[0]
            file_id = f"{month}/{file}"

            # 使用 Dash 特殊標記 `download://` 來表示下載
            link = f"[{display_name}](download://{file_id})"
            links.append(link)

    return "  \n".join(links)

df["File"] = df["File"].apply(generate_file_links)



@app.callback(
    Output("download-component", "data"),
    Input("table", "active_cell"),
    prevent_initial_call=True
)
def trigger_download(active_cell):
    if not active_cell:
        return dash.no_update

    row = active_cell["row"]
    col = active_cell["column_id"]

    if col != "File":
        return dash.no_update

    file_string = df.at[row, "File"]
    if "download://" in file_string:
        file_path = file_string.split("(download://")[1].split(")")[0]  # 解析下載路徑
        file_path = f"../../data_source/{file_path}"

        if os.path.exists(file_path):
            return dcc.send_file(file_path)

    return dash.no_update


## 2. 使用 Markdown 並攔截點擊事件

1. 在 dash_table.DataTable 的 File 欄位顯示 Markdown 連結

  - 202502_A.txt → [A.txt](/202502_A.txt)
  - 202502_B.xlsx → [B.xlsx](/202502_B.xlsx)
  - 這樣表格內的 File 欄位會顯示藍字、底線的超連結。
2. 監聽 active_cell，偵測點擊的檔案名稱

  - 透過 ctx.triggered 確認是哪個超連結被點擊。
3. 解析 File 欄位內的 Markdown 連結

  - 取得 file_id（檔案名稱），並組合出完整的 file_path。
4. 透過 dcc.Download 下載檔案

  - 確保檔案存在後，執行 dcc.send_file(file_path) 讓使用者下載。

In [ ]:
def generate_file_links(file_string):
    if not file_string:
        return ""

    files = file_string.split(', ')
    links = []

    for file in files:
        parts = file.split('_', 1)
        if len(parts) > 1:
            display_name = parts[1]  # 顯示在表格內的文字
            month = parts[0]  # 取得月份資訊
            file_id = f"{month}/{file}"  # 生成唯一檔案識別碼

            # 這裡產生一個 Markdown 連結，href 設為特殊識別格式
            link = f"[{display_name}](/{file_id})"
            links.append(link)

    return " \n".join(links)  # 使用 Markdown 換行符號，讓多個檔案顯示在不同行

df["File"] = df["File"].apply(generate_file_links)


## 因為 dash_table.DataTable 沒有內建 n_clicks, 改用 active_cell 來偵測使用者點擊哪個欄位：
@app.callback(
    Output("download-component", "data"),
    Input("table", "active_cell"),
    prevent_initial_call=True
)
def trigger_download(active_cell):
    if not active_cell:
        return dash.no_update

    row = active_cell["row"]
    col = active_cell["column_id"]

    if col != "File":
        return dash.no_update  # 確保只有點擊 "File" 欄位時才觸發下載

    file_string = df.at[row, "File"]  # 取得被點擊的檔案欄位值
    file_links = file_string.split("  \n")  # Markdown 轉換後的多個連結

    # 從回調內容取得觸發的超連結
    ctx = dash.callback_context
    if not ctx.triggered:
        return dash.no_update

    # 解析被點擊的檔案名稱
    triggered_text = ctx.triggered[0]["value"]
    matched_files = [link for link in file_links if triggered_text in link]

    if matched_files:
        file_id = matched_files[0].split('](')[1].rstrip(')')  # 取得 "/{file_id}"
        file_path = f"../../data_source{file_id}"

        if os.path.exists(file_path):
            return dcc.send_file(file_path)

    return dash.no_update


## 3. 使用 / </a/> html 的/ download

### 3-1

In [ ]:
####### 使用真實路徑
import dash
from dash import dcc, html, dash_table
import pandas as pd

app = dash.Dash(__name__)

# 原始數據
data = [
    [1, "H02U", 'DH240P23.00', 23, '202502_A.txt, 202502_B.xlsx'],
    [2, "H02D", 'DH240P11.00', 23, '202502_C.png, 202502_D.csv'],
    [3, "H03F", 'DH240P10.00', 21, '202502_CD.xlsx, 202502_E.csv, 2025-02_F.csv'],
    [4, "D03F", 'CH240P10.00', 11, '']
]

# 轉換為 DataFrame
df = pd.DataFrame(data, columns=["Issue Number", "Chart Name", "Lot", "Count", "File"])

# 產生下載的<a>標籤，這次直接使用真實路徑
def generate_file_links(file_string):
    if not file_string:
        return ""

    files = file_string.split(', ')
    links = []

    for file in files:
        parts = file.split('_', 1)
        if len(parts) > 1:
            display_name = parts[1]  # 取 _ 之後的字眼
            month = parts[0]  # 取 _ 之前的月份字串
            file_path = f"../../data_source/update/{month}/{file}"  # 生成真實檔案路徑

            # 直接在 href 中使用真實路徑
            link = f'<a href="/{file_path}" download>{display_name}</a>'
            links.append(link)

    return "  \n".join(links)  # 使用換行顯示多個檔案

df["File"] = df["File"].apply(generate_file_links)

app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[
            {"name": col, "id": col, "presentation": "html"} if col == "File" else {"name": col, "id": col}
            for col in df.columns
        ],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
    ),
])

if __name__ == '__main__':
    app.run_server(debug=True)


### 3-2

1. 檔案路徑更新：在 download_file 函數中，file_path 被更新為 os.path.join(base_path, file_path)，其中 base_path 設為 ../../data_source/update，並且將 file_path 拼接進來，這樣就能正確定位到檔案的根目錄。

2. 下載檔案：當使用者點擊下載連結時，會根據正確的路徑提供檔案下載。

In [ ]:
######## 使用 flask
import dash
from dash import dcc, html, dash_table, Input, Output
from flask import send_file  # 這行是為了導入 send_file
import pandas as pd
import os

app = dash.Dash(__name__)

# 原始數據
data = [
    [1, "H02U", 'DH240P23.00', 23, '202502_A.txt, 202502_B.xlsx'],
    [2, "H02D", 'DH240P11.00', 23, '202502_C.png, 202502_D.csv'],
    [3, "H03F", 'DH240P10.00', 21, '202502_CD.xlsx, 202502_E.csv, 2025-02_F.csv'],
    [4, "D03F", 'CH240P10.00', 11, '']
]

# 轉換為 DataFrame
df = pd.DataFrame(data, columns=["Issue Number", "Chart Name", "Lot", "Count", "File"])

# 產生下載的<a>標籤
def generate_file_links(file_string):
    if not file_string:
        return ""

    files = file_string.split(', ')
    links = []

    for file in files:
        parts = file.split('_', 1)
        if len(parts) > 1:
            display_name = parts[1]  # 取 _ 之後的字眼
            month = parts[0]  # 取 _ 之前的月份字串
            file_id = f"{month}/{file}"  # 生成下載 ID

            # 使用 <a> 標籤並加入 download 屬性
            link = f'<a href="/download/{file_id}" download>{display_name}</a>'
            links.append(link)

    return "  \n".join(links)  # 使用換行顯示多個檔案

df["File"] = df["File"].apply(generate_file_links)

app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[
            {"name": col, "id": col, "presentation": "html"} if col == "File" else {"name": col, "id": col}
            for col in df.columns
        ],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
    ),
    dcc.Download(id="download-component")
])

# 提供檔案下載
@app.server.route('/download/<path:file_path>')
def download_file(file_path):
    # 更新檔案路徑，根據你提供的結構
    base_path = '../../data_source/update'
    file_path = os.path.join(base_path, file_path)  # 假設文件存放在指定的根目錄
    if os.path.exists(file_path):
        return send_file(file_path, as_attachment=True)
    else:
        return "File not found", 404

if __name__ == '__main__':
    app.run_server(debug=True)


@app.server.route('/download/<path/:file_path>')

這段程式碼表示，當用戶訪問 /download/<file_path> 路徑時（例如 /download/?202502_A.txt），Flask 會觸發 download_file 函數，並將 file_path 參數傳遞給它。

- <path/:file_path>：這是一個動態路由參數，表示 URL 中的檔案路徑。Flask 會從 URL 中提取 file_path，並將它傳遞給 download_file 函數。

- 例如，當你點擊表格中的某個檔案下載鏈接 "/download?/202502_A.txt" 時，file_path 的值會是 202502_A.txt，並且此檔案會在你的伺服器上被查找並提供下載。

#### 補充

In [ ]:
import dash
from dash import dcc, html, dash_table, Input, Output, State
import pandas as pd
import os
import flask

app = dash.Dash(__name__)
server = app.server  # 讓 Flask 處理靜態檔案

# 設定文件目錄
FILE_DIRECTORY = "../../data_source/update/2025/02/"

# 原始數據
data = [
    [1, "H02U", 'DH240P23.00', 23, '202502_A.txt'],
    [2, "H02D", 'DH240P11.00', 23, '202502_C.png'],
    [3, "H03F", 'DH240P10.00', 21, '202502_D.xlsx'],
    [4, "D03F", 'CH240P10.00', 11, '']
]

# 轉換為 DataFrame
df = pd.DataFrame(data, columns=["Issue Number", "Chart Name", "Lot", "Count", "File"])

# 生成下載連結函數
#(超連結樣式：檔案名稱用 <a> 標籤包裹，並加上 style="color:blue; text-decoration:underline;"，讓它顯示為藍色底線文字。)
def create_download_link(file_name):
    if file_name:  # 檢查檔案名是否為空
        return f'<a href="/download/{file_name}" style="color:blue; text-decoration:underline;">{file_name}</a>'
    return ""

# 在 DataFrame 中添加超連結欄位
df["Download"] = df["File"].apply(create_download_link)

app.layout = html.Div([
    html.H3("檔案下載表"),
    dash_table.DataTable(
        id='table',
        columns=[
            {"name": col, "id": col, "presentation": "markdown" if col == "Download" else "text"}
            for col in df.columns
        ],
        data=df.to_dict('records'),
        style_cell={'textAlign': 'center', 'fontSize': 16},
        style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
        style_data_conditional=[
            {"if": {"column_id": "Download"}, "textDecoration": "underline", "color": "blue"}
        ],
        markdown_options={"html": True},  # 允許HTML標記
    )
])

# Flask 伺服器處理檔案下載
@app.server.route("/download/<file_name>")
def download_file(file_name):
    file_path = os.path.join(FILE_DIRECTORY, file_name)
    if os.path.exists(file_path):
        return flask.send_file(file_path, as_attachment=True)
    else:
        return "File not found", 404

if __name__ == "__main__":
    app.run(debug=True)


### 若一個cell一個檔案

In [ ]:
import dash
from dash import dcc, html, dash_table, Input, Output, State
import pandas as pd
import os
import flask

app = dash.Dash(__name__)
server = app.server  # 讓 Flask 處理靜態檔案

# 設定文件目錄
FILE_DIRECTORY = "../../data_source/update/2025/02/"

# 原始數據
data = [
    [1, "H02U", 'DH240P23.00', 23, '202502_A.txt'],
    [2, "H02D", 'DH240P11.00', 23, '202502_C.png'],
    [3, "H03F", 'DH240P10.00', 21, '202502_D.xlsx'],
    [4, "D03F", 'CH240P10.00', 11, '']
]

# 轉換為 DataFrame
df = pd.DataFrame(data, columns=["Issue Number", "Chart Name", "Lot", "Count", "File"])

app.layout = html.Div([
    html.H3("點擊檔案名稱自動下載"),

    dash_table.DataTable(
        id='table',
        columns=[{"name": col, "id": col} for col in df.columns],
        data=df.to_dict('records'),
        style_cell={'textAlign': 'center', 'fontSize': 16},
        style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
        style_data_conditional=[
            {"if": {"column_id": "File"}, "textDecoration": "underline", "color": "blue", "cursor": "pointer"}
        ],  # 讓"File"欄位有藍色底線並可點擊
    ),

    dcc.Download(id="download-file"),  # 用於處理檔案下載
])

# 監聽點擊事件
@app.callback(
    Output("download-file", "data"),
    Input("table", "active_cell"),
    State("table", "data"),
    prevent_initial_call=True
)
def download_file(active_cell, data):
    if active_cell and active_cell["column_id"] == "File":  # 確保點擊的是 "File" 欄位
        row = active_cell["row"]  # 獲取點擊的行索引
        file_name = data[row]["File"]  # 獲取對應的檔案名稱

        if file_name:  # 確保檔案名稱不是空的
            file_path = os.path.join(FILE_DIRECTORY, file_name)
            if os.path.exists(file_path):
                return dcc.send_file(file_path)  # 直接下載檔案
    return dash.no_update  # 沒點擊或檔案不存在時不做任何動作

if __name__ == "__main__":
    app.run(debug=True)


1. active_cell 會返回使用者點擊的儲存格資訊，例如

    {'row': 2, 'column': 4, 'column_id': 'File'}
2. 取得點擊的 行索引 (row)
  - 讀取 data[row]["File"] 來獲取檔名
  - 使用 dcc.send_file(file_path) 讓 Dash 直接下載檔案（不開新分頁！）

3. style_data_conditional 設定 "File" 欄位：
  - 藍色文字,底線
  - 鼠標變成手型（cursor: "pointer"），告訴使用者可以點擊。

 點擊 Dash Table 裡的檔案名稱 → 直接下載檔案 🎉
✅ 不會開啟新分頁 🚀
✅ 如果欄位是空的，不會發生任何動作